In [1]:
import os
from dotenv import load_dotenv
from funzioni_albero_2025 import pausa,operazione_ERP,eliminazione_ERP,modifica_ERP

from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langgraph.prebuilt import ToolNode,create_react_agent
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationTokenBufferMemory

In [2]:
load_dotenv()
api_key = os.getenv("openai_key")

In [3]:
llm = ChatOpenAI(
    api_key=api_key,
    model='gpt-4o',
    temperature=0
)

In [4]:
#memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True)
memory = ConversationTokenBufferMemory(
    memory_key='chat_history',
    return_messages=True,
    llm=llm,
    max_token_limit=1000
    )

C:\Users\andre\AppData\Local\Temp\ipykernel_1220\3669997139.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(


In [5]:
def query_memory(input_text: str, memory=None):
    """
    Funzione per interpretare le richieste dell'utente relative al contesto.
    Ora riceve direttamente l'oggetto memory come parametro.
    """
    if memory is None:
        return input_text
        
    # Carica la chat history dalla memoria
    memory_vars = memory.load_memory_variables({})
    chat_history = memory_vars.get("chat_history", [])
    
    template = PromptTemplate.from_template(
        """
        Il tuo compito è RIFORMULARE la richiesta corrente dell'utente in una frase ESEGUIBILE, sostituendo riferimenti vaghi con informazioni specifiche dalla chat history.

        NON ANALIZZARE la chat history. NON DESCRIVERE cosa è stato fatto.
        INVECE, RIFORMULA la richiesta corrente in una nuova frase completa e autosufficiente.

        ESEMPI CORRETTI:
        Storia: "aggiungi due ore al foglio ore di oggi"
        Richiesta: "cancella l'ultima operazione"
        ✓ Output: "togli due ore al foglio ore di oggi"

        Storia: "registra check-in ore 9"
        Richiesta: "modifica l'orario"
        ✓ Output: "modifica l'orario del check-in di oggi dalle ore 9"

        ESEMPI ERRATI:
        Storia: "aggiungi due ore al foglio ore"
        Richiesta: "cancella l'ultima operazione"
        ✓ Output: "togli due ore al foglio ore"
        ✗ NO: "L'ultima operazione eseguita è stata l'aggiunta di due ore"
        ✗ NO: "Si procede a cancellare l'ultima operazione che era..."

        REGOLE:
        1. Inizia SEMPRE con un verbo d'azione
        2. Includi TUTTI i dettagli specifici dalla chat history
        3. La frase deve essere ESEGUIBILE come comando
        4. NON usare mai frasi descrittive o analitiche

        chat_history: {chat_history}
        richiesta corrente: {domanda}

        Riformula la richiesta in una singola frase eseguibile:
        """
    )
    
    chain_memory = template | llm
    print("memory")
    print(chain_memory.invoke({"domanda": input_text, "chat_history": chat_history}).content)
    
    
    return chain_memory.invoke({"domanda": input_text, "chat_history": chat_history}).content

In [6]:
tools = [
    Tool(
        name="query_memory",
        func=lambda x: query_memory(x, memory),
        description="""
            Usa questo tool SOLO quando:
            - La richiesta contiene riferimenti vaghi ("quello", "questa", "precedente", "ultima")
            - Serve contestualizzare la richiesta con informazioni dalla chat history
            IMPORTANTE: Questo tool RIFORMULA la richiesta in un comando eseguibile
            
            Esempio INPUT: "cancella l'ultima operazione"
            Esempio OUTPUT: "togli 2 ore al foglio ore di oggi"
        """
    ),
    Tool(
        name="pausa",
        func=pausa,
        description="""
            Usa questo tool SOLO per:
            - Registrare check-in o check-out
            - Gestire pause pranzo
            - Registrare timbrature di entrata/uscita
            Non usare per:
            - Modificare timbrature esistenti (usa operazione_ERP)
            - Altre operazioni non legate a timbrature
            
            Esempio: "devo timbrare l'entrata" -> usa questo tool
            Esempio: "modifica la timbratura di ieri" -> usa operazione_ERP
        """
    ),
    Tool(
        name="operazione_ERP",
        func=operazione_ERP,
        description="""
            Usa questo tool per:
            - Creare nuovi record (richieste ferie, note spese, eventi in calendario ecc.)
            - Modificare record esistenti di qualsiasi tipo
            - Aggiornare informazioni nel sistema

            Non usare per:
            - Eliminare record (usa eliminazione_ERP)
            - Timbrature semplici (usa pausa)

            Esempio: "voglio prendere ferie lunedì" -> usa questo tool
            Esempio: "modifica l'importo della nota spese" -> usa questo tool
            Esempio: "togli due ore al foglio ore" -> usa questo tool
            Esempio: "crea un evento in calendario ..." -> usa questo tool
        """
    ),
    Tool(
        name="eliminazione_ERP",
        func=eliminazione_ERP,
        description="""
            Usa questo tool ESCLUSIVAMENTE per:
            - Eliminare definitivamente record dal sistema
            - Cancellare completamente una richiesta o un documento
            Usa SOLO quando l'utente richiede esplicitamente di:
            - Cancellare
            - Eliminare
            - Rimuovere
            Non usare mai per:
            - Modifiche o aggiornamenti (usa operazione_ERP)
            - Disattivazioni temporanee

            Esempio: "elimina la nota spese di ieri" -> usa questo tool
            Esempio: "rimuovi la richiesta ferie" -> usa questo tool
            Esempio: "togli un'ora dal foglio ore di oggi" --> non usare questo tool
        """
    ),
    Tool(
        name="modifica_ERP",
        func=modifica_ERP,
        description="""
            Usa questo tool esclusivamente per modificare i record del db 
            Usa SOLO quando l'utente richiede esplicitamente di:
            - Modificare
            - Cambiare
            - Spostare

            Esempio: "sposta la riunione di oggi a domani" -> usa questo tool
            Esempio: "cambia la data del meeting.." -> usa questo tool
            Esempio: "Elimina il meeting di oggi" -> non usare questo tool
        """
    )
]

tool_node = ToolNode(tools)

agent = create_react_agent(
    model=llm,
    tools=tool_node,
    store=memory
)

In [7]:
def invoke_agent(agent, user_input: dict, memory):
    """
    Invoca l'agente con la gestione corretta della memoria.
    """
    # Estrae il messaggio dell'utente
    user_message = user_input.get("messages", [])[-1][1] if user_input.get("messages") else ""
    
    
    # Prepara l'input per l'agent
    agent_input = {
        "messages": [{"role": "user", "content": user_message}]
    }
    
    # Invoca l'agent
    response = agent.invoke(agent_input)
    
    # Aggiorna la memoria con la nuova interazione
    memory.save_context(
        {"input": user_message},
        {"output": response.get("output", "")}
    )
    
    return response

In [17]:
inputs = {
        "messages": [("user", "cancellalo")]
    }
    
risposta = invoke_agent(agent,inputs,memory)

memory
Cancella l'evento in calendario con nome FNS programmato per lunedì 13 gennaio.


In [18]:
print(risposta['messages'][-1].content)

L'evento in calendario con nome "FNS" programmato per lunedì 13 gennaio è stato cancellato con successo.


In [19]:
risposta

{'messages': [HumanMessage(content='cancellalo', additional_kwargs={}, response_metadata={}, id='541a7441-a4e1-404a-979f-0def86821dda'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sN7gbwhyFBeJRuBlHO80MFhS', 'function': {'arguments': '{"__arg1":"cancellalo"}', 'name': 'query_memory'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 744, 'total_tokens': 763, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7d65f1a5b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bd95c1cd-d602-47e3-a0ff-fa795527520c-0', tool_calls=[{'name': 'query_memory', 'args': {'__arg1': 'cancellalo'}, 'id': 'call_sN7gbwhyFBeJRuBlHO80MFhS', 'type': 'tool_call'}], usage_metadata={'input_tokens': 

In [ ]:
inputs = {
        "messages": [("user", "elimina l'ultima operazione")]
    }
    
risposta = invoke_agent(agent,inputs,memory)

In [ ]:
print(risposta['messages'][-1].content)

In [ ]:
risposta

In [ ]:
memory.load_memory_variables({})